In [1]:
%load_ext autoreload

In [2]:
import os
import sys

sys.path.append('../src-py/')

In [3]:
from datasets import load_dataset, load_metric

from transformers import BartTokenizer, BartForConditionalGeneration
from mt_bart import BartForMultiTaskArgGeneration

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [5]:
%autoreload

from utils import *

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#### Load models and testing data:

In [7]:
from argparse import Namespace

In [8]:
#model_path = '../data/output/model/v1_epoch_2'
model_path = '../data/output/model/v1_epoch_2'
baseline_model_path = '../data/output/model/baseline_epoch_2'

In [9]:
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForMultiTaskArgGeneration.from_pretrained(model_path, without_classification_head=False).to(device)
baseline_model = BartForMultiTaskArgGeneration.from_pretrained(baseline_model_path, without_classification_head=True).to(device)

True
False


In [17]:
args = Namespace(batch_size= 8,
    max_target_length= 100,
    max_source_length=500,
    ignore_pad_token_for_loss=False,
    distributed=False,
    num_beams=20,
    num_test_samples=1000
)

test_loader, raw_data = get_test_loader('../data/valid.json', tokenizer, args)

100%|██████████| 1000/1000 [00:04<00:00, 222.93it/s]


In [18]:
#Evalute ROUGE scores
gen_kwargs = {
    "max_length": args.max_target_length,
    "num_beams": args.num_beams,
    "do_sample": True, 
    "max_length":100,
    "top_p":0.92, 
    "top_k":50
}

In [19]:
eval_scores, generated_attacks, gt_attacks, detected_weak_premises = generate_and_evaluate_text(model, tokenizer, test_loader, args, gen_kwargs)
baseline_eval_scores, baseline_generated_attacks, _, _ = generate_and_evaluate_text(baseline_model, tokenizer, test_loader, args, gen_kwargs)

In [13]:
print(baseline_eval_scores)

{'rouge1': 18.2195, 'rouge2': 2.5213, 'rougeL': 11.5149, 'rougeLsum': 15.2488, 'bert-fscore': 0.03}


In [14]:
print(eval_scores)

{'rouge1': 15.7547, 'rouge2': 1.6485, 'rougeL': 10.5031, 'rougeLsum': 13.4869, 'bert-fscore': 0.04}


In [63]:
raw_data[0]

{'conclusion': " Pink is just light red and doesn't deserve it's own name.",
 'premises': ["i understand the need for more precise naming when you're doing something artsy.",
  "what i'm focusing on is how we describe colors in everyday life.",
  'pink is just a light shade of the color red.',
  'we already have a ton of precedence for just calling something a light or dark color.',
  'when people see a light or dark blue object, they just call it light blue or dark blue.',
  'when the see a light or dark green thing, they call it light green or dark green.',
  'for some reason, when we see something that is light red we call it pink.',
  "i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name."],
 'weak_premises': ['when people see a light or dark blue object, they just call it light blue or dark blue.'],
 'counter': 'this is incorrect. russians have special purpose words for light and dark blue, for example. color words are a cultural linguistic

In [64]:
df = pd.DataFrame(list(zip([x['conclusion'] for x in raw_data], [' '.join(x['premises']) for x in raw_data], [' '.join(x['weak_premises']) for x in raw_data], gt_attacks, baseline_generated_attacks, generated_attacks)),
                  columns=['conclusion', 'arguments', 'weak_premises', 'gt_attacks', 'baseline_gen_attacks', 'gen_attacks'])

In [65]:
df.to_csv('../data/output/test_preds.csv')

In [66]:
df.sample(10).head(n=100)

conclusion  \
260                                                                                    Owning a pet is like owning a slave or a prisoner   
844   The following law I propose, restricting a restaurant's ability to "run out" of menu items and to close, would be good for America   
634                                                                                            Fortnite Battle Royale is not a good game   
891                                     Bitcoin and similar cryptos are bad currencies, bad investments and terrible for the environment   
104                                                                                                               I dislike Led Zeppelin   
995                                            LinkedIn is a waste of time and not really useful for career development and finding jobs   
991                                                                                     Throwing away perfectly good food is not immoral   
295                                                                   Banning extremist political groups from Reddit is often a bad idea   
20                                                                   Elon Musk should not be selling flamethrowers as if they were toys.   
710                                                                               Lethal Injection should be replaced by a firing squad.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [67]:
def evaluate_gen_attacks(generated_attacks, gt_attacks):
    metric = load_metric('rouge')
    bertscore_metric = load_metric('bertscore')
    
    metric.add_batch(predictions=generated_attacks, references=gt_attacks)
    bertscore_metric.add_batch(predictions=generated_attacks, references=gt_attacks)
    
    result = metric.compute(use_stemmer=True)
    bertscore_result = bertscore_metric.compute(lang='en', rescale_with_baseline=True)

    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    result['bert-fscore'] = round(np.mean(bertscore_result['f1']), 2)
    
    return result

In [68]:
print(evaluate_gen_attacks(df.gen_attacks, df.weak_premises))

{'rouge1': 16.7183, 'rouge2': 6.728, 'rougeL': 13.895, 'rougeLsum': 14.4025, 'bert-fscore': 0.1}


In [69]:
print(evaluate_gen_attacks(df.baseline_gen_attacks, df.weak_premises))

{'rouge1': 19.5, 'rouge2': 8.0788, 'rougeL': 15.6733, 'rougeLsum': 16.4782, 'bert-fscore': 0.11}
